<a href="https://colab.research.google.com/github/RomuloRangel/projeto2023_02/blob/main/Projeto2023_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agrupamento para definir quais os locais e horarios Que mais ocorreram os tiroteios

Link do datset : https://catalog.data.gov/dataset/nypd-shooting-incident-data-historic

# Importando as bibliotecas necessarias

In [68]:
import pandas as pd
import numpy as np

In [69]:
#Definirei uma importação via URL
url = (' https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD')
df = pd.read_csv(url)

In [70]:
df.describe()

,INCIDENT_KEY,PRECINCT,JURISDICTION_CODE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
count,2.731200e+04,27312.000000,27310.000000,2.731200e+04,27312.000000,27302.000000,27302.000000
mean,1.208605e+08,65.635362,0.326913,1.009449e+06,208127.401608,40.737892,-73.909051
std,7.341286e+07,27.305705,0.737743,1.837783e+04,31886.377757,0.087525,0.066272
min,9.953245e+06,1.000000,0.000000,9.149281e+05,125756.718750,40.511586,-74.249303
25%,6.386088e+07,44.000000,0.000000,1.000029e+06,182834.335938,40.668481,-73.943027
50%,9.037222e+07,68.000000,0.000000,1.007731e+06,194486.570312,40.700318,-73.915221
75%,1.888102e+08,81.000000,0.000000,1.016838e+06,239518.468750,40.824067,-73.882330
max,2.611902e+08,123.000000,2.000000,1.066815e+06,271127.687500,40.910818,-73.702046


In [71]:
print(df.isna().sum())

INCIDENT_KEY                   0
OCCUR_DATE                     0
OCCUR_TIME                     0
BORO                           0
LOC_OF_OCCUR_DESC          25596
PRECINCT                       0
JURISDICTION_CODE              2
LOC_CLASSFCTN_DESC         25596
LOCATION_DESC              14977
STATISTICAL_MURDER_FLAG        0
PERP_AGE_GROUP              9344
PERP_SEX                    9310
PERP_RACE                   9310
VIC_AGE_GROUP                  0
VIC_SEX                        0
VIC_RACE                       0
X_COORD_CD                     0
Y_COORD_CD                     0
Latitude                      10
Longitude                     10
Lon_Lat                       10
dtype: int64


#TRATAMENTO E TRANSFORMAÇÃO DOS DADOS

Limpeza das colunas que nao impactam no modelo positivamente

In [72]:
#Armazenei as features que possuiam mais de 70% dos dados faltantes em uma lista depois exclui pois nao serveriam para o modelo
un_information = ['LOC_OF_OCCUR_DESC','LOC_CLASSFCTN_DESC','LOCATION_DESC','PERP_AGE_GROUP','PERP_SEX','PERP_RACE' ,'VIC_AGE_GROUP','Lon_Lat']
df_edit1 = df.drop(un_information,axis=1)

#Tratamento de valores nulos

In [73]:
print(df_edit1.isna().sum())

INCIDENT_KEY                0
OCCUR_DATE                  0
OCCUR_TIME                  0
BORO                        0
PRECINCT                    0
JURISDICTION_CODE           2
STATISTICAL_MURDER_FLAG     0
VIC_SEX                     0
VIC_RACE                    0
X_COORD_CD                  0
Y_COORD_CD                  0
Latitude                   10
Longitude                  10
dtype: int64


In [74]:
df_edit1

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,STATISTICAL_MURDER_FLAG,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,228798151,05/27/2021,21:30:00,QUEENS,105,0.0,False,M,BLACK,1.058925e+06,180924.000000,40.662965,-73.730839
1,137471050,06/27/2014,17:40:00,BRONX,40,0.0,False,M,BLACK,1.005028e+06,234516.000000,40.810352,-73.924942
2,147998800,11/21/2015,03:56:00,QUEENS,108,0.0,True,M,WHITE,1.007668e+06,209836.531250,40.742607,-73.915492
3,146837977,10/09/2015,18:30:00,BRONX,44,0.0,False,M,WHITE HISPANIC,1.006537e+06,244511.140625,40.837782,-73.919457
4,58921844,02/19/2009,22:58:00,BRONX,47,0.0,True,M,BLACK,1.024922e+06,262189.406250,40.886238,-73.852910
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27307,245029823,05/14/2022,03:02:00,BRONX,48,0.0,False,M,BLACK,1.011526e+06,247828.000000,40.846864,-73.901413
27308,239583450,01/22/2022,13:15:00,MANHATTAN,30,0.0,False,M,WHITE HISPANIC,9.974580e+05,240485.000000,40.826743,-73.952273
27309,246825728,06/18/2022,03:29:00,MANHATTAN,32,0.0,False,M,BLACK,1.000999e+06,234464.000000,40.810209,-73.939496
27310,246876579,06/19/2022,20:08:00,BRONX,46,2.0,False,M,WHITE HISPANIC,1.012980e+06,251028.000000,40.855644,-73.896141


Importando bibliotecas para tratamento de valores ausentes (NAN)

In [75]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer



In [76]:
#Usando pacote para tratar valores NAN definindo os padroes entre frequencia , media, constante
usando_frequencia = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
usando_media = SimpleImputer(missing_values=np.nan, strategy='mean')
usando_constante = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='0')
#Usando para dados mais proximos ou seja para dados que estão mais proximos dos valores ausentes
usando_proximo = KNNImputer(n_neighbors=2)

In [78]:
#Para a feature JURISDICTION_CODE sera usado mair frequencia por ser um dado categorico
df_edit1.JURISDICTION_CODE = usando_frequencia.fit_transform(df_edit1[['JURISDICTION_CODE' ]])
#Para as features Latitude Longetude e Lon_lat irei usar dados mais proximos por se tratar de uma coordenada
df_edit1.Latitude = usando_proximo.fit_transform(df_edit1[['Latitude']])
df_edit1.Longitude = usando_proximo.fit_transform(df_edit1[['Longitude']])

#Apos devidos tratmentos de valores ausentes
print(df_edit1.isna().sum())

INCIDENT_KEY               0
OCCUR_DATE                 0
OCCUR_TIME                 0
BORO                       0
PRECINCT                   0
JURISDICTION_CODE          0
STATISTICAL_MURDER_FLAG    0
VIC_SEX                    0
VIC_RACE                   0
X_COORD_CD                 0
Y_COORD_CD                 0
Latitude                   0
Longitude                  0
dtype: int64
